Install dependencies

In [107]:
%pip install opencv-python keras numpy matplotlib --user

Note: you may need to restart the kernel to use updated packages.


Import dependencies

In [108]:
import numpy as np
import cv2
import os
import random
import matplotlib.pyplot as plt
import pickle

Setup Dataset directory and category

In [109]:
Directory = r'H:\Drowsiness Detection\dataset\train'
CATEGORIES = ['Closed' , 'Open', 'Yawn', 'No_yawn']

Convert dataset images to array

In [110]:
img_size = 24
data = []

for category in CATEGORIES:
    folder = os.path.join(Directory,category)
    label = CATEGORIES.index(category)
    for img in os.listdir(folder):
        img_path = os.path.join(folder, img)
        img_arr = cv2.imread(img_path)
        img_arr = cv2.cvtColor(img_arr, cv2.COLOR_BGR2GRAY)
        img_arr = cv2.resize(img_arr,(img_size, img_size),1)
        data.append([img_arr , label])
        
#Verify length of data
len(data)

2900

Data preprocessing

In [111]:
random.shuffle(data)

feat = []
lbl = []

for features,label in data:
    feat.append(features)
    lbl.append(label)

In [112]:
#covert them into array:
feat = np.array(feat)
lbl = np.array(lbl)

In [113]:
# save the data into system:
pickle.dump(feat , open('feat.pkl' , 'wb'))
pickle.dump(lbl , open('lbl.pkl' , 'wb'))

In [114]:
# normalize the image array:
feat = feat/255

In [115]:
feat

array([[[0.59215686, 0.59215686, 0.58431373, ..., 0.41960784,
         0.41960784, 0.40784314],
        [0.56862745, 0.56862745, 0.56470588, ..., 0.41568627,
         0.41568627, 0.41568627],
        [0.69803922, 0.67843137, 0.62745098, ..., 0.41960784,
         0.41568627, 0.40392157],
        ...,
        [0.17647059, 0.19607843, 0.46666667, ..., 0.40392157,
         0.51764706, 0.82352941],
        [0.38823529, 0.41960784, 0.29803922, ..., 0.37647059,
         0.37647059, 0.48235294],
        [0.37254902, 0.43529412, 0.31372549, ..., 1.        ,
         0.58431373, 0.3254902 ]],

       [[0.72156863, 0.7254902 , 0.5372549 , ..., 0.62745098,
         0.62745098, 0.62745098],
        [0.74509804, 0.52941176, 0.50980392, ..., 0.62745098,
         0.63137255, 0.62352941],
        [0.5372549 , 0.44313725, 0.57254902, ..., 0.61960784,
         0.62352941, 0.61568627],
        ...,
        [0.64313725, 0.6627451 , 0.6745098 , ..., 0.61176471,
         0.64313725, 0.67058824],
        [0.6

In [116]:
# reshape the feat array to (24,24,1)
img_rows,img_cols = 24,24
feat = feat.reshape(feat.shape[0],img_rows,img_cols,1)
feat.shape

(2900, 24, 24, 1)

In [117]:
# we will be using keras to create the model:
from keras.models import Sequential
from keras.layers import Conv2D , MaxPooling2D , Flatten , Dense

In [118]:
# creating model:
model = Sequential()

model.add(Conv2D(64 , (3,3) , activation = 'relu' , input_shape= feat.shape[1:]))
model.add(MaxPooling2D((1,1)))

model.add(Conv2D(64 , (3,3) , activation = 'relu'))
model.add(MaxPooling2D((1,1)))

model.add(Conv2D(64 , (3,3) , activation = 'relu'))
model.add(MaxPooling2D((1,1)))

model.add(Flatten())

model.add(Dense(128, activation = 'relu'))

model.add(Dense(4, activation = 'softmax'))

In [119]:
# compile model that we have created
model.compile(optimizer = 'adam' , loss = 'sparse_categorical_crossentropy' , metrics = ['accuracy'])

In [120]:
# fit feat , lbl to the model to see accuracy of model:
model.fit(feat, lbl, epochs = 10 , validation_split = 0.1 , batch_size = 32)

Epoch 1/10


82/82 ━━━━━━━━━━━━━━━━━━━━ 3s 29ms/step - accuracy: 0.5875 - loss: 0.8947 - val_accuracy: 0.7897 - val_loss: 0.4549
Epoch 2/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - accuracy: 0.8192 - loss: 0.3731 - val_accuracy: 0.8552 - val_loss: 0.3469
Epoch 3/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - accuracy: 0.8661 - loss: 0.2992 - val_accuracy: 0.8586 - val_loss: 0.2725
Epoch 4/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - accuracy: 0.8956 - loss: 0.2514 - val_accuracy: 0.8759 - val_loss: 0.2630
Epoch 5/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - accuracy: 0.9264 - loss: 0.1784 - val_accuracy: 0.9103 - val_loss: 0.2058
Epoch 6/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - accuracy: 0.9468 - loss: 0.1369 - val_accuracy: 0.9034 - val_loss: 0.1967
Epoch 7/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - accuracy: 0.9675 - loss: 0.1016 - val_accuracy: 0.9276 - val_loss: 0.2034
Epoch 8/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - accuracy: 0.9666 - loss: 0.0910 - val_accuracy: 0.9276 - val_loss: 0.

In [121]:
# save model and architecture to single file
model.save("drowsiness.keras")